In [5]:
print("-----------------------------------")
print("---------- INSERT CSV -------------")
print("-----------------------------------")

-----------------------------------
---------- INSERT CSV -------------
-----------------------------------


In [6]:
import pandas as pd

inF = open(input("Enter Filename: "),'r')
pd.set_option('display.max_columns', 46)
#orders = pd.read_csv('upselling1.csv', dtype='unicode')
orders = pd.read_csv(inF, dtype='unicode')

Enter Filename: upselling2.csv


In [7]:
orders.columns = ['Purchase_Order_Number', 'Sold_To_Party_Name', 'Material', 'Order_Quantity_(Item)']

In [8]:
orders['Purchase_Order_Number']= orders['Purchase_Order_Number'].fillna('NULL')
orders['Sold_To_Party_Name']= orders['Sold_To_Party_Name'].fillna('NULL')
orders['Material']= orders['Material'].fillna('NULL')
orders['Order_Quantity_(Item)']= orders['Order_Quantity_(Item)'].fillna('NULL')



In [9]:
# orders.drop(columns=['Key','Plan', 'Sales_document', 'Sales_Doc_Item', 'Sales_Document_Type', 'Order_Quantity_(Item)', 'Sales_document_type', 'Purchase_Order_Number','Delivery_block'
# ,'Reason_for_Rejection',
# 'Created_on_Month_(Fiscal)',
# 'Price_Group',
# 'CSR_Gensuite',
# 'Lab_Office',
# 'Cost_from_SAP',
# 'Subtotal_1_From_Sap',
# 'Subtotal_2_From_Sap',
# 'Subtotal_3_From_SAP',
# 'Subtotal_4_From_SAP',
# 'Tax_amount_(ext)_From_SAP',
# 'Total',
# 'Cost(extended)',
# 'Subtotal_1_Gross_Value(ext)',
# 'Subtotal_2_After_discount_(Ext)',
# 'Subtotal_3_Total_After_Freight_(Ext)',
# 'Subtotal_4_Freight_Charges_or_Min_Ord_Qty',
# 'Tax_amount(ext)',
# 'Sold_To_Party1',
# 'Sold_To_Party_Name1',
# 'Order_Status'])
# orders = orders[1:]

In [10]:
grouped_for_editing = orders.groupby(["Purchase_Order_Number", "Sold_To_Party_Name" , "Material"])["Purchase_Order_Number"].count()

In [11]:
grouped_for_editing.to_csv('groups.csv', index=True, header = True, sep=',')

In [12]:
calculate_percents = pd.read_csv('groups.csv', dtype='unicode', sep=',', engine='python')

In [13]:
calculate_percents["counts_of_order"] = calculate_percents.groupby('Purchase_Order_Number')['Material'].transform('count')
calculate_percents["counts_of_material"]= calculate_percents.groupby('Material')['Purchase_Order_Number'].transform('count')


In [14]:
calculate_percents_materials = calculate_percents.groupby(["Material", "Purchase_Order_Number"])["Purchase_Order_Number"].size().reset_index(name="qty")



In [ ]:
calculate_percents_materials.to_csv('groupsMaterials.csv', index=True, header = True, sep=',')
df_of_materials = calculate_percents_materials[['Material', 'qty']]
df_of_materials2 = df_of_materials.drop_duplicates()
df_of_materials2 =  df_of_materials2.drop_duplicates()

#
#          THIS IS THE NEWEST CODE CURRENTLY WORKING ON 
#                          061818
#
import numpy as np
#Finding the matching positions to materials
#Going through each material to find locations of the orders
i = 0
count = 0
total_count_of_each_part =0

for index in range (0, len(df_of_materials2.index)):
    count = 0
    master_df = pd.DataFrame(columns = ['Material', 'count'])
    df = pd.DataFrame(columns = ['Material', 'Purchase_Order_Number']) #Data frame for getting all purchases
    df3 = pd.DataFrame(columns = ['Material']) #Dataframe for getting all parts
    material_to_check = df_of_materials2.iloc[i].Material #calling on the first material in the list of materials with no duplicates
    list_of_values = []
    #for index in calculate_percents_materials: *FOR LOOP not needed*
    print("Material: " + material_to_check)  
    #print("     Orders material is in:")
        #checking where material is equal to get true and false locations
    boolean_percent = calculate_percents_materials["Material"] == material_to_check
        
        #get list of true locations to get the data
    list_of_index = np.where(boolean_percent)[0]
    
   
        #check to see if the list is more than 1 location
   # if len(list_of_index) > 1:
          
            #if more than 1 location append all the materials to a dataframe
    z = 0;
    max_counts_parts = pd.DataFrame(columns = ['Material', 'qty']) #
    for x in list_of_index:
                
                #This is where I need to get all the materials from the orders.
                get_materials_of_order = calculate_percents_materials.loc[x].Purchase_Order_Number
                print("\tOrder #: " + calculate_percents_materials.loc[x].Purchase_Order_Number)
                #Getting all the materials per order
                get_all_materials_per_order = calculate_percents_materials["Purchase_Order_Number"] == get_materials_of_order 
                get_all_materials_per_order_index = np.where(get_all_materials_per_order)[0]
                df_all_parts_of_order = pd.DataFrame(columns = ['Material', 'qty'])
                print("\t\tParts of Order: ")
                #Getting a df of all the parts of this order
                for p in get_all_materials_per_order_index:
                    
                    part_need = calculate_percents_materials.loc[p]
                    part_material = part_need.Material
                    part_qty = part_need.qty
                    addme = pd.DataFrame([part_material, part_qty])
                    addme = addme.T
                    addme.columns = ['Material', 'qty']
                    df_all_parts_of_order = df_all_parts_of_order.append(addme, ignore_index=True )
                    #print("\t\tPart: " + part_need.Material)
                    
                    
                
                #Append all the part of that order to the master
                master_df = master_df.append(df_all_parts_of_order, ignore_index=True)
                max_counts_parts = max_counts_parts.append(master_df, ignore_index=True)
                #max_counts_parts.columns = ['Material', 'qty']
                
                #max_counts_parts = max_counts_parts.replace({'count':{'nan':np.nan}}).dropna(subset=['count'], inplace=True)
                
                #Get the counts of all the parts in the order
                display_master_df = master_df.groupby(['Material']).size().reset_index(name="qty")
                display_master_df.columns = ['Material', 'qty']
                
                #Find the percents of each of the parts based off the counts
                for q in range(0, len(display_master_df.index)):
                    get_value = display_master_df.loc[q].Material
                    
                    quantity_of_item = int (display_master_df.loc[q].qty)
                    get_value_index = df_of_materials["Material"] == get_value
                    get_value_position_array = np.where(get_value_index)[0]
#                     print('Array of positions of material in df_of_materials2')
#                     print(get_value_position_array)
                    #print("Length of array; " + str(len(get_value_position_array)))
                    get_value_position = get_value_position_array[0]
#                     print("Value Position: " + str(get_value_position))
#                     print("Size of df_of_materials2" + str(len(df_of_materials2.index)))
                    total_count_prop = df_of_materials2.loc[get_value_position]
                   
                    total_count = len(get_value_position_array)
                    
                    #print("Total QTY: " + str(total_count))
                    if(get_value != material_to_check):
                      
                        value_st = ((quantity_of_item / total_count) * 100)
#                     print ("Display master qty: " + str(display_master_df.loc[q].count))
                        if(value_st > 49):
                            print ( "\t\tPart:" + str(display_master_df.loc[q].Material) + "\t\t" + str(value_st) + "%")
                       # else:
                            #print ("\t\tPercents too low for upselling")
                    
                        
                
                #variable names for appending the dataframes
                material_name = calculate_percents_materials.loc[x].Material
                purchase_order_name = calculate_percents_materials.loc[x].Purchase_Order_Number
                count_var = (calculate_percents_materials.loc[x].qty)
                              
                df2 = pd.DataFrame([material_name, purchase_order_name])
                #had to transpose the df to get the columns
                df2 = df2.T
                df2.columns = ['Material', 'Purchase_Order_Number']
                df = df.append(df2)
               

            
    i = i +1  

Material: 000020V025
	Order #: K059776
		Parts of Order: 
	Order #: K072066
		Parts of Order: 
	Order #: K072775
		Parts of Order: 
	Order #: K519272
		Parts of Order: 


C:\Users\Public\Documents\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


		Part:773590002		100.0%
	Order #: K521311
		Parts of Order: 
		Part:773590002		100.0%
	Order #: WP16885
		Parts of Order: 
		Part:773590002		100.0%
		Part:MS90727-121		100.0%
	Order #: i001482
		Parts of Order: 
		Part:773590002		100.0%
		Part:MS90727-121		100.0%
Material: 000021V118
	Order #: K068980
		Parts of Order: 
	Order #: K069178
		Parts of Order: 
Material: 000021V163
	Order #: K065284
		Parts of Order: 
	Order #: K066810
		Parts of Order: 
	Order #: K067567
		Parts of Order: 
		Part:010876V007		100.0%
		Part:MS90727-119		50.0%
		Part:MS90727-164		100.0%
	Order #: K067640
		Parts of Order: 
		Part:010876V007		100.0%
		Part:054473V		50.0%
		Part:MS90727-119		50.0%
		Part:MS90727-164		100.0%
	Order #: K070620
		Parts of Order: 
		Part:010876V007		100.0%
		Part:054473V		66.66666666666666%
		Part:MS90727-115		66.66666666666666%
		Part:MS90727-119		50.0%
		Part:MS90727-164		100.0%
	Order #: K519719
		Parts of Order: 
		Part:010876V007		100.0%
		Part:054473V		83.33333333333334%
		P

	Order #: K061695
		Parts of Order: 
	Order #: K063858
		Parts of Order: 
	Order #: K067366
		Parts of Order: 
Material: 000046V
	Order #: K062629
		Parts of Order: 
	Order #: K063623
		Parts of Order: 
		Part:025950V		100.0%
		Part:026160V002		100.0%
	Order #: WP14339
		Parts of Order: 
		Part:025950V		100.0%
		Part:026160V002		100.0%
Material: 000049V002
	Order #: s023162
		Parts of Order: 
		Part:039891V010		100.0%
		Part:071889V001		100.0%
		Part:S23162		100.0%
Material: 000050V
	Order #: K523088
		Parts of Order: 
		Part:027260V002		100.0%
Material: 000057V037
	Order #: K522448
		Parts of Order: 
Material: 000058V003
	Order #: K069878
		Parts of Order: 
Material: 000058V004
	Order #: K523632
		Parts of Order: 
Material: 000058V006
	Order #: I003509
		Parts of Order: 
	Order #: K063210
		Parts of Order: 
	Order #: K068341
		Parts of Order: 
Material: 000058V008
	Order #: K520060
		Parts of Order: 
Material: 000072F
	Order #: K518576
		Parts of Order: 
Material: 000075F
	Order #: K0

	Order #: WP16310
		Parts of Order: 
		Part:005881V158		100.0%
		Part:082613V072		100.0%
	Order #: wp13247
		Parts of Order: 
		Part:005881V158		100.0%
		Part:082613V072		100.0%
Material: 000220V
	Order #: I003727
		Parts of Order: 
	Order #: K060562
		Parts of Order: 
	Order #: K064001
		Parts of Order: 
		Part:MS51960-66NS		100.0%
	Order #: K064107
		Parts of Order: 
		Part:MS51960-66NS		100.0%
	Order #: K064393
		Parts of Order: 
		Part:MS51960-66NS		100.0%
	Order #: K065030
		Parts of Order: 
		Part:024035V003		50.0%
		Part:MS51960-66NS		100.0%
	Order #: K065176
		Parts of Order: 
		Part:024035V003		50.0%
		Part:MS51960-66NS		100.0%
	Order #: K067035
		Parts of Order: 
		Part:024035V003		50.0%
		Part:038279F		100.0%
		Part:058574V		50.0%
		Part:MS51960-66NS		100.0%
	Order #: K072783
		Parts of Order: 
		Part:024035V003		50.0%
		Part:038279F		100.0%
		Part:058574V		50.0%
		Part:MS51960-66NS		100.0%
	Order #: K072963
		Parts of Order: 
		Part:024035V003		50.0%
		Part:038279F		100.0%


...

		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: K068170
		Parts of Order: 
		Part:026575V		100.0%
		Part:027143V064		100.0%
		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: K068282
		Parts of Order: 
		Part:026575V		100.0%
		Part:027143V064		100.0%
		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: K068639
		Parts of Order: 
		Part:026575V		100.0%
		Part:027143V064		10

		Part:026575V		100.0%
		Part:027143V064		100.0%
		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: K518274
		Parts of Order: 
		Part:026575V		100.0%
		Part:027143V064		100.0%
		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: K519613
		Parts of Order: 
		Part:026575V		100.0%
		Part:027143V064		100.0%
		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: K519918
		Parts of Ord

		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: K522619
		Parts of Order: 
		Part:001150V002		55.633802816901415%
		Part:026575V		100.0%
		Part:027143V064		100.0%
		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:058224V054		100.0%
		Part:066382V003		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: K522678
		Parts of Order: 
		Part:001150V002		55.633802816901415%
		Part:026575V		100.0%
		Part:027143V064		100.0%
		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:058224V054		100.0%
		Part:066382V003		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0

		Part:MS51959-80NS		100.0%
	Order #: WP13658
		Parts of Order: 
		Part:001150V002		64.7887323943662%
		Part:026575V		100.0%
		Part:027143V064		100.0%
		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:058224V054		100.0%
		Part:066382V003		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: WP13959
		Parts of Order: 
		Part:001150V002		65.49295774647888%
		Part:026575V		100.0%
		Part:027143V064		100.0%
		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:058224V054		100.0%
		Part:066382V003		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: WP14475
		Parts of Order: 
		Part:001150V002		66.19718309859155%
		Part:026575V		100.0%
		Part:027143V064		100.0%

		Part:054783V018		100.0%
		Part:058224V054		100.0%
		Part:066382V003		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: WP16309
		Parts of Order: 
		Part:001150V002		76.05633802816901%
		Part:026575V		100.0%
		Part:027143V064		100.0%
		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:058224V054		100.0%
		Part:066382V003		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100.0%
		Part:080275V		100.0%
		Part:081050V		100.0%
		Part:MS51959-80NS		100.0%
	Order #: WP16449
		Parts of Order: 
		Part:001150V002		76.76056338028168%
		Part:026575V		100.0%
		Part:027143V064		100.0%
		Part:027760V009		50.0%
		Part:043058F		50.0%
		Part:054378A276		100.0%
		Part:054783V018		100.0%
		Part:058224V054		100.0%
		Part:066382V003		100.0%
		Part:070840F001		50.0%
		Part:070841F001		50.0%
		Part:078718V003		100